In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union

getcontext().prec = 30
scale_factor = 1

class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(center_x)
        self.center_y = Decimal(center_y)
        self.angle = Decimal(angle)

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (Decimal('0.0') * scale_factor, tip_y * scale_factor),
            (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

def get_tree_list_side_length(tree_list):
    all_polygons = [t.polygon for t in tree_list]
    bounds = unary_union(all_polygons).bounds
    return Decimal(max(bounds[2] - bounds[0], bounds[3] - bounds[1])) / scale_factor

def calculate_score(csv_path):
    try:
        df = pd.read_csv(csv_path)
        df['x'] = df['x'].astype(str).str.strip().str.lstrip('s')
        df['y'] = df['y'].astype(str).str.strip().str.lstrip('s')
        df['deg'] = df['deg'].astype(str).str.strip().str.lstrip('s')
        df[['group_id', 'item_id']] = df['id'].str.split('_', n=2, expand=True)
        
        total_score = Decimal('0')
        for group_id, group_data in df.groupby('group_id'):
            tree_list = [ChristmasTree(center_x=row['x'], center_y=row['y'], angle=row['deg'])
                         for _, row in group_data.iterrows()]
            side = get_tree_list_side_length(tree_list)
            n = int(group_id)
            total_score += side ** 2 / Decimal(str(n))
        return float(total_score)
    except Exception as e:
        return f"Error: {e}"

print("Score calculator ready")

Score calculator ready


In [2]:
# Check scores of pre-optimized solutions
import os

preopt_dir = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/'
csv_files = [
    preopt_dir + 'best_ensemble.csv',
    preopt_dir + 'ensemble.csv',
    preopt_dir + 'santa-2025.csv',
    preopt_dir + 'submission.csv',
]

for f in csv_files:
    if os.path.exists(f):
        score = calculate_score(f)
        print(f"{os.path.basename(f)}: {score}")

best_ensemble.csv: 70.67610239809181


ensemble.csv: 70.67610239809181


santa-2025.csv: 70.67610239809181


submission.csv: 70.67650087844204


In [3]:
# Check more solutions
import glob

# Check bucket-of-chump
bucket_dir = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/bucket-of-chump/'
for f in glob.glob(bucket_dir + '*.csv'):
    score = calculate_score(f)
    print(f"{os.path.basename(f)}: {score}")

# Check telegram
telegram_dir = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/telegram/'
for f in glob.glob(telegram_dir + '*.csv'):
    score = calculate_score(f)
    print(f"{os.path.basename(f)}: {score}")

submission.csv: 70.67650087844204


72.49.csv: 72.4957386818876


71.97.csv: 71.97202711347292


In [4]:
# Check santa25-public for better solutions
santa25_dir = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa25-public/'
for f in sorted(glob.glob(santa25_dir + '*.csv')):
    score = calculate_score(f)
    print(f"{os.path.basename(f)}: {score}")

New_Tree_144_196.csv: 72.92792008327766


santa2025_ver2_v61.csv: 72.95192515723619


santa2025_ver2_v63.csv: 72.94742695347861


santa2025_ver2_v65.csv: 72.93529377404569


santa2025_ver2_v66.csv: 72.93859945313127


...

submission_JKoT1.csv: 72.489482836154


submission_JKoT2.csv: 72.4893480678526


submission_JKoT3.csv: 72.48948787611938


submission_JKoT4.csv: 72.48950405756999


submission_opt1.csv: 70.99069202345483


In [5]:
# Check the experiments directory for better solutions
exp_dir = '/home/nonroot/snapshots/santa-2025/21116303805/code/experiments/'
for root, dirs, files in os.walk(exp_dir):
    for f in files:
        if f.endswith('.csv'):
            path = os.path.join(root, f)
            score = calculate_score(path)
            print(f"{path.replace(exp_dir, '')}: {score}")

004_sa_v1_parallel/submission_best.csv: 70.67610239809181


004_sa_v1_parallel/submission_v18.csv: 70.67610239808829


005_backward_propagation/submission.csv: 70.67610239809181


005_backward_propagation/submission_v21.csv: 70.67610239809181


005_backward_propagation/optimized.csv: 70.67610239809181


002_preoptimized/submission.csv: 70.67610239461068


In [6]:
# Check the main submission.csv in the code directory
main_sub = '/home/nonroot/snapshots/santa-2025/21116303805/code/submission.csv'
score = calculate_score(main_sub)
print(f"Main submission.csv: {score}")

# Check submission_candidates
candidates_dir = '/home/nonroot/snapshots/santa-2025/21116303805/code/submission_candidates/'
for f in sorted(glob.glob(candidates_dir + '*.csv')):
    score = calculate_score(f)
    print(f"{os.path.basename(f)}: {score}")

Main submission.csv: 70.67610239809181


candidate_000.csv: 150.80978013947083


candidate_001.csv: 70.67609930809661


candidate_002.csv: 70.67610239809181


candidate_003.csv: 70.67610239809181


candidate_004.csv: 70.67610239809181


In [7]:
# Check the latest snapshots for better solutions
latest_snapshots = [
    '/home/nonroot/snapshots/santa-2025/21145965159/',
    '/home/nonroot/snapshots/santa-2025/21145968755/',
    '/home/nonroot/snapshots/santa-2025/21145966992/',
]

for snap in latest_snapshots:
    for root, dirs, files in os.walk(snap):
        for f in files:
            if f.endswith('.csv') and 'sample' not in f:
                path = os.path.join(root, f)
                score = calculate_score(path)
                print(f"{path}: {score}")